In [20]:
import openai
import pandas as pd
import json
import os
from PyPDF2 import PdfReader

# Set the OpenAI API key
openai.api_key = 'your_openai_api_key_here'

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

# Function to evaluate job descriptions using ChatGPT API
def evaluate_job_description(job_desc, positive_examples, resume_text=None):
    prompt = (
        "You are an assistant that predicts the likelihood of getting a recruiter call "
        "based on past successful job descriptions and a candidate's resume.\n\n"
    )
    
    if resume_text:
        prompt += f"Resume:\n{resume_text}\n\n"

    prompt += (
        "Positive Examples:\n"
        + "\n\n".join(positive_examples)
        + "\n\nBelow is a new job description. Evaluate whether this job matches the positive examples "
        "and the resume on a scale of 0 to 1 (0 = no match, 1 = high match).\n"
        f"Job Description:\n{job_desc}\n\nOutput only the prediction value."
    )

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Replace with your model, if needed
            messages=[{"role": "user", "content": prompt}]
        )

        prediction = float(response['choices'][0]['message']['content'].strip())
        return prediction
    except Exception as e:
        print(f"Error with OpenAI API: {e}")
        return None

# Load job data from CSV
def load_job_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading job data: {e}")
        return None

# Main function
def main():
    # Load the transformed job data
    file_path = "data/job_data_transformed.csv"
    job_data = load_job_data(file_path)
    
    if job_data is None:
        return

    # Ensure the dataset has a 'job_description' column
    if 'job_description' not in job_data.columns:
        print("The dataset does not contain a 'job_description' column.")
        return

    # Extract positive examples from PDFs
    positive_examples = []
    pdf_files = ["data/positiveexample1.pdf", "data/positiveexample2.pdf"]
    for pdf in pdf_files:
        text = extract_text_from_pdf(pdf)
        if text:
            positive_examples.append(text.strip())

    if not positive_examples:
        print("No positive examples could be loaded from the PDFs.")
        return

    # Optionally include resume text
    resume_path = "data/resume.pdf"  # Update with your resume file path
    resume_text = extract_text_from_pdf(resume_path) if os.path.exists(resume_path) else None

    # List to store jobs that you should apply for
    jobs_to_apply = []

    # Limit processing to the first 10 jobs
    job_data = job_data.head(10)

    # Iterate through each job description and evaluate
    for index, row in job_data.iterrows():
        job_desc = row['job_description']
        prediction = evaluate_job_description(job_desc, positive_examples, resume_text)

        if prediction is not None and prediction > 0.7:  # Threshold for strong match
            job_entry = row.to_dict()
            job_entry['job_url'] = row.get('job_url', 'N/A')  # Add job_url if available
            jobs_to_apply.append(job_entry)
            print(f"Job URL: {job_entry['job_url']}")  # Output job_url to console

    # Save the results
    if jobs_to_apply:
        results_df = pd.DataFrame(jobs_to_apply)
        results_df.to_csv("jobs_to_apply.csv", index=False)
        print("Jobs to apply for saved to jobs_to_apply.csv.")
    else:
        print("No jobs meet the threshold to apply.")

if __name__ == "__main__":
    main()



Job URL: https://www.themuse.com/jobs/leidos/program-manager-c35bbd
Job URL: https://www.themuse.com/jobs/thermofisherscientific/staff-program-manager-372992
Job URL: https://www.themuse.com/jobs/jpmorganchase/lead-technical-program-manager-de0e4f
Jobs to apply for saved to jobs_to_apply.csv.
